# 応用情報工学演習：Scikit-Learnマスターコース
## 実践編 配布用コードファイル

#### 基本ライブラリの読み込み

In [1]:
import os, sys
import glob

import warnings
warnings.simplefilter('ignore')

import numpy as np
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

np.random.seed(1)

#### データ読み込み関数

In [2]:
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#                               このセルは変更を禁止します                                  #
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#-------------------------------------------------------------------------------------------#
#-                                  データ読み込み関数                                     -#
#-------------------------------------------------------------------------------------------#
def load_data_(root_dir='../data/gw_dataset'):
    """
    グループワーク用　学習用データ読み込み関数

    Args:
        * root_dir: 配布したgw_datasetへのパスを指定する (default: '../data/gw_dataset')
        
    Output:
        * X_train:  正解ラベル付き学習用画像   shape: (920x3,072)
                    1行あたり、画像1枚の画素値を格納したnp.array
                    画像920枚、各画像32x32=3,072次元のベクトル
                    
        * y_train:  X_trainの正解ラベル        shape: (920, )
                    クラス数は46　920枚分のクラスラベル（0～45）の整数が格納されている
        
        * X_trainu: 正解ラベルなし学習用画像   shape: (3,680x3,072)
                    形式はX_trainと同じ
                    
        * X_tval:   正解ラベル付き検証用画像   shape: (230x3,072)
                    形式はX_trainと同じ
                    
        * y_val:    X_valの正解ラベル          shape: (230, )
                    形式はy_valと同じ                    
    """
    
    #----------------- 学習用データの読み込み -------------------#
    train_paths = glob.glob(os.path.join(root_dir, "train", "*.png"))
    X_train = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in train_paths])
    y_train = np.load(os.path.join(root_dir, "y_train.npy"))

    #------------ ラベルなし学習用データの読み込み --------------#
    trainu_paths = glob.glob(os.path.join(root_dir, "train-u", "*.png"))
    X_trainu = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in trainu_paths])
    
    #----------------- 検証用データの読み込み -------------------#
    val_paths = glob.glob(os.path.join(root_dir,"val", "*.png"))
    X_val = np.array([np.array(Image.open(p)).astype(np.float32).ravel() for p in val_paths])
    y_val = np.load(os.path.join(root_dir, "y_val.npy"))
    
    return X_train, y_train, X_trainu, X_val, y_val


In [3]:
### データ読み込み関数の実行
X_train, y_train, X_trainu, X_val, y_val = load_data_() # すべて必要な場合
# print(X_train.shape, y_train.shape, X_trainu.shape, X_val.shape, y_val.shape)

#X_train, y_train, _, X_val, y_val = load_data_() # いらないデータがある場合
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

FileNotFoundError: [Errno 2] No such file or directory: '../data/gw_dataset\\y_train.npy'

#### 評価関数

In [4]:
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#                               このセルは変更を禁止します                                  #
#-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-#
#-------------------------------------------------------------------------------------------#
#-                                       評価関数                                          -#
#-------------------------------------------------------------------------------------------#

def eval_(y_pred):
    """
    グループワーク用　評価関数
    実行する前にy_valがglobalスコープに読み込まれている必要がある

    Args:
        * y_pred:   識別結果　y_valと同形式・同shapeでなければならない
    """

    try:
        y_val
    except NameError as e:
        print("y_valが読み込まれていません")
        
    assert y_pred.shape == y_val.shape, 'y_predとy_valのサイズが一致しません'
    
    print("valデータでの識別精度:{0:.3f}".format(accuracy_score(y_val, y_pred)))

In [5]:
#y=np.append(y_train, -1, axis=0)
ys_train=np.full(3680, -1)
#for num in range(3680):
#y=np.insert(y_train, 920+num, -1)

In [6]:
y=np.append(y_train, ys_train, axis=0)

In [7]:
x=np.append(X_train, X_trainu, axis=0)

In [9]:
#Self-Trainingによる半教師あり学習
import pandas as pd
from sklearn.svm import SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


# # ベース識別器であるsvm.SVCオブジェクトを作成
svc = SVC(kernel='rbf', C=10, probability=True)
# ### Self-Training
clf = SelfTrainingClassifier(base_estimator=svc, criterion='k_best', k_best=300) # オブジェクト作成 確信度上位15件に疑似ラベルを振る
clf.fit(x, y) 
#clf.fit(X_train, y_train)       # fit
y_pred = clf.predict(X_val)     # prediction

# def eval_knn(X_train, y_train, X_test, y_test, k=10):
#     clf = KNeighborsClassifier(n_neighbors=k)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
    
#     return y_pred

#y_pred = eval_knn(X_train, y_train, X_val, y_val)

eval_(y_pred) # 評価関数の実行（y_valが読み込まれている必要がある）

valデータでの識別精度:0.817
